In [1]:
# libraries
import pandas as pd
import numpy as np
import time
import torch
from tqdm import tqdm
from preprocessing import extract_data_mooc, extractFeatures,extractItemUserId,extractNextStateItem,extractNextUserState,UserNextInteraction, delta, t_batch_update
from model import RODIE,dynamic_embedding
from train import train_rodie

In [ ]:
## Téléchargement des données
!wget https://snap.stanford.edu/data/act-mooc.tar.gz
!tar -xzf  act-mooc.tar.gz
!mkdir data

In [ ]:
features = pd.read_csv("act-mooc/mooc_action_features.tsv",sep="\t")
labels = pd.read_csv("act-mooc/mooc_action_labels.tsv",sep="\t")
users = pd.read_csv("act-mooc/mooc_actions.tsv",sep="\t")

#### Load & Preprocess Data

In [ ]:
mooc_data = extract_data_mooc()

delta_u  = delta(mooc_data.copy(),"user_id")
delta_i  = delta(mooc_data.copy(),"item_id")
nextItemInteraction = UserNextInteraction(mooc_data.copy())
next_state_user = extractNextUserState(mooc_data.copy())


mooc_data['delta_u'] = delta_u
mooc_data['delta_i'] = delta_i
mooc_data['nextItemInteraction'] = nextItemInteraction
mooc_data['next_state_user'] = next_state_user

data = mooc_data.copy()
data = data[ (data.nextItemInteraction != -1) | (data.next_state_user != -1)  ]

data = data[['user_id', 'item_id', 'timestamp', 'state_label','delta_u', 'delta_i', 'nextItemInteraction', 'next_state_user','f1', 'f2', 'f3','f4']]
data.head()

#### T-batches

In [ ]:
t_batches = t_batch_update(data.reset_index())

##### Initialize Device

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = "cpu"
print('Using device:', device)

##### Initialize the model

In [ ]:
embedding_dim = 32
data_torch = torch.from_numpy(data.values.astype(np.float32))
U_dynamic,I_dynamic = dynamic_embedding(data_torch)  # Initial dynamic embedding
    
U_dynamic = U_dynamic.to(device)
I_dynamic = I_dynamic.to(device)

model = RODIE(embedding_dim,data_torch).to(device)

##### Train Loop

In [ ]:
n_epochs = 3
lambda_u = 1e-3
lambda_i = 1e-3
optimizer = torch.nn.Adam(model.parameters(),lr=1e-3)

model_ = train_rodie(t_batches,
          U_dynamic,
          I_dynamic,
          optimizer,
          n_epochs,
          lambda_u,
          lambda_i
          )